In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from pyspark.sql import SparkSession

In [3]:
spark.sparkContext.setLogLevel("ERROR")


In [4]:
spark = SparkSession.builder.appName("KMeansexample").getOrCreate()

In [5]:
print(spark.version)


3.5.3


In [ ]:
data = [(1, "foo"), (2, "bar"), (3, "baz")]
df = spark.createDataFrame(data, ["id", "value"])
df.show()

In [ ]:
df = pd.read_csv("smoking_driking_dataset_Ver01.csv.gz")

In [ ]:
df

# Data Preparation
## Check for Missing Values

In [ ]:
df.isnull().sum()

## Convert Object Data types into Numerical Data types

In [ ]:
if "sex" in df.columns:
    one_hot = pd.get_dummies(df["sex"], prefix = 'Is_')
    df_ = pd.concat([df, one_hot], axis = 1)
    df = df_.drop(["sex"], axis = 1)

In [ ]:
df

In [ ]:
if "DRK_YN" in df.columns:    
    df = df.drop(["DRK_YN"], axis = 1)

    

## Standardising

In [ ]:
pipeline = Pipeline([
    ('standardiser', StandardScaler())
])

In [ ]:
Standard_df = pipeline.fit_transform(df)

In [ ]:
Standard_df

# Model

## Rough Model

In [ ]:
kmeans = KMeans(n_clusters =5,  n_init = 10, random_state = 42)
kmeans_pred = kmeans.fit_predict(Standard_df)
wcss = kmeans.inertia_

In [ ]:
centroids = kmeans.cluster_centers_

In [ ]:
plt.figure(figsize = (8,10))

plt.scatter(Standard_df[:,-1], Standard_df[:, -2], c = kmeans_pred, cmap = "viridis", alpha = 0.6)
plt.scatter(centroids[:,0], centroids[:, 1], c = "red", marker = 'X')

### The graph seems cluttered so Decided to try plot with the sample.

In [ ]:
np.random.seed(42)
sample_indices = np.random.choice(Standard_df.shape[0], replace = False, size = 1000)
sample_df = Standard_df[sample_indices]
sample_preds = kmeans_pred[sample_indices]

In [ ]:
plt.figure(figsize = (8,10))

plt.scatter(sample_df[:,2], sample_df[:, 8], c = sample_preds, cmap = "viridis", alpha = 0.6)
plt.scatter(centroids[:,0], centroids[:, 1], c = "red", marker = 'X')


In [ ]:
silhouette_score(Standard_df, kmeans_pred)